In [1]:
# Imports
import os
import datetime
import json
from typing import overload, Any, List, Dict, Tuple, Set, Sequence, Union
from numpy import *
import pandas as pd
import pyopenms as oms
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm
import shutil
import requests
from sklearn.impute import KNNImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

# Ignore seaborn warning for future deprecation of module part
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# import methods from FIA python script
from FIA import *

In [2]:
def sirius_integration(sirius_path: Union[str, oms.String], feature_xml_path: str, msfile_path:str, csifingerid_path:str, experiment_path:str, experiment:oms.MSExperiment=None,
                       email:str=None, password:str=None,
                       filter_by_num_masstraces:int=1, precursor_mz_tolerance:float=10.0, precursor_mz_tolerance_unit:str="ppm", precursor_rt_tolerance:float=5.0,
                       isotope_pattern_iterations:int=3, feature_only:str="false", maxmz:int=-1) -> None:
    """
    Preprocess and export .ms files for SIRIUS
    """
    experiment = load_experiment(experiment_path, experiment)
    clean_dir(msfile_path)
    clean_dir(csifingerid_path)

    saa = oms.SiriusAdapterAlgorithm()
    param = saa.getDefaults()
    param.setValue("preprocessing:filter_by_num_masstraces", filter_by_num_masstraces)
    param.setValue("preprocessing:precursor_mz_tolerance", precursor_mz_tolerance)
    param.setValue("preprocessing:precursor_mz_tolerance_unit", precursor_mz_tolerance_unit)
    param.setValue("preprocessing:precursor_rt_tolerance", precursor_rt_tolerance)
    param.setValue("preprocessing:isotope_pattern_iterations", isotope_pattern_iterations)
    param.setValue("preprocessing:feature_only", feature_only)
    param.setValue("project:maxmz", maxmz)

    saa.determineSiriusExecutable(sirius_path)
    saa. logInSiriusAccount(sirius_path, email, password)

    feature_mapping = oms.FeatureMapping_FeatureToMs2Indices()
    fm_info = oms.FeatureMapping_FeatureMappingInfo()
    saa.preprocessingSirius(feature_xml_path, experiment, fm_info, feature_mapping)

    tmp_ms_file = clean_dir(feature_xml_path, "tmp_ms")
    tmp_out_dir = clean_dir(feature_xml_path, "tmp")
    v_cmpinfo = saa.callSiriusQProcess( tmp_ms_file, tmp_out_dir, sirius_path, csifingerid_path, False)

    smsf = oms.SiriusMSFile()
    smsf.store(experiment, msfile_path, feature_mapping, feature_only, isotope_pattern_iterations, False, v_cmpinfo) 	

In [8]:
s = oms.String(r"C:\Program Files\OpenMS-3.1.0\share\OpenMS\THIRDPARTY\Sirius\sirius.bat")
print(s)

C:\Program Files\OpenMS-3.1.0\share\OpenMS\THIRDPARTY\Sirius\sirius.bat


In [3]:

sirius_integration(oms.String(r"C:\Program Files\OpenMS-3.1.0\share\OpenMS\THIRDPARTY\Sirius\sirius.bat"), r"..\runs\example\features_rtaligned\mg1655_P3-A8_neg.featureXML", r"..\runs\example\mss",
                   r"..\runs\example\csifingerids", r"..\data\example data\exampleA_ecolistrains\mg1655_P3-A8_neg.mzXML", experiment=None, email=None, password=None,
                   filter_by_num_masstraces=1, precursor_mz_tolerance=10.0, precursor_mz_tolerance_unit="ppm", precursor_rt_tolerance=5.0,
                   isotope_pattern_iterations=3, feature_only="false", maxmz=-1)

AssertionError: arg executable wrong type

In [2]:
sa = oms.SiriusAdapterAlgorithm()
par = sa.getParameters()
print_params(par)

Param: b'read_sirius_stdout' Value: false Description: Read and print the standard output and error of the Sirius executable, even if it succeeds.
Param: b'preprocessing:filter_by_num_masstraces' Value: 1 Description: Number of mass traces each feature has to have to be included. To use this parameter, setting the feature_only flag is necessary
Param: b'preprocessing:precursor_mz_tolerance' Value: 10.0 Description: Tolerance window for precursor selection (Feature selection in regard to the precursor)
Param: b'preprocessing:precursor_mz_tolerance_unit' Value: ppm Description: Unit of the precursor_mz_tolerance
Param: b'preprocessing:precursor_rt_tolerance' Value: 5.0 Description: Tolerance window (left and right) for precursor selection [seconds]
Param: b'preprocessing:isotope_pattern_iterations' Value: 3 Description: Number of iterations that should be performed to extract the C13 isotope pattern. If no peak is found (C13 distance) the function will abort. Be careful with noisy data -

## Loading
Reads in files

In [6]:
experiment_path = "../data/example data/exampleA_ecolistrains/mg1655_P3-A8_neg.mzXML"
experiment = read_experiment(experiment_path)

In [ ]:
def separate_paired_array(paired_arr):
    size = comb(len(paired_arr), 2, exact=True)
    arr = np.fromiter(chain.from_iterable(combinations(paired_arr, 2)), dtype=np.dtype( (float, 2) ), count=size).reshape(-1, 2, 2)
    return np.squeeze(np.hsplit(arr, 2))